<a href="https://colab.research.google.com/github/takky0330/NLP/blob/master/TableParser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/google-research/tapas/blob/master/notebooks/sqa_predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2020 The Google AI Language Team Authors

Licensed under the Apache License, Version 2.0 (the "License");

In [1]:
# Copyright 2019 The Google AI Language Team Authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

Running a Tapas fine-tuned checkpoint
---
This notebook shows how to load and make predictions with TAPAS model, which was introduced in the paper: [TAPAS: Weakly Supervised Table Parsing via Pre-training](https://arxiv.org/abs/2004.02349)

# Clone and install the repository


First, let's fetch the code from the github repository and install it

In [2]:
! git clone https://github.com/google-research/tapas.git

Cloning into 'tapas'...
remote: Enumerating objects: 44, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 280 (delta 11), reused 19 (delta 4), pack-reused 236
Receiving objects: 100% (280/280), 271.79 KiB | 5.13 MiB/s, done.
Resolving deltas: 100% (134/134), done.


In [3]:
! pip install ./tapas

Processing ./tapas
     |████████████████████████████████| 3.5MB 1.6MB/s 
     |████████████████████████████████| 10.1MB 16.6MB/s 
     |████████████████████████████████| 516.2MB 16kB/s 
     |████████████████████████████████| 716kB 25.9MB/s 
     |████████████████████████████████| 3.5MB 22.1MB/s 
     |████████████████████████████████| 358kB 45.0MB/s 
     |████████████████████████████████| 61kB 5.4MB/s 
     |████████████████████████████████| 1.2MB 35.7MB/s 
     |████████████████████████████████| 153kB 44.5MB/s 
     |████████████████████████████████| 51kB 4.5MB/s 
     |████████████████████████████████| 225kB 44.1MB/s 
     |████████████████████████████████| 63.2MB 87kB/s 
     |████████████████████████████████| 81kB 4.4MB/s 
     |████████████████████████████████| 122kB 38.0MB/s 
     |████████████████████████████████| 153kB 50.4MB/s 
     |████████████████████████████████| 440kB 42.7MB/s 
     |████████████████████████████████| 235kB 39.5MB/s 
     |██████████████████████████████

# Fetch models fom Google Storage

Next we can get pretrained checkpoint from Google Storage. For the sake of speed, this is base sized model trained on [SQA](https://www.microsoft.com/en-us/download/details.aspx?id=54253). Note that best results in the paper were obtained with with a large model, with 24 layers instead of 12.

In [4]:
! gsutil cp gs://tapas_models/2020_04_21/tapas_sqa_base.zip . && unzip tapas_sqa_base.zip

Copying gs://tapas_models/2020_04_21/tapas_sqa_base.zip...
/ [1 files][  1.0 GiB/  1.0 GiB]   48.4 MiB/s                                   
Operation completed over 1 objects/1.0 GiB.                                      
Archive:  tapas_sqa_base.zip
   creating: tapas_sqa_base/
  inflating: tapas_sqa_base/model.ckpt.data-00000-of-00001  
  inflating: tapas_sqa_base/model.ckpt.index  
  inflating: tapas_sqa_base/README.txt  
  inflating: tapas_sqa_base/vocab.txt  
  inflating: tapas_sqa_base/bert_config.json  
  inflating: tapas_sqa_base/model.ckpt.meta  


# Imports

In [5]:
import tensorflow.compat.v1 as tf
import os 
import shutil
import csv
import pandas as pd
import IPython

tf.get_logger().setLevel('ERROR')

In [6]:
from tapas.utils import tf_example_utils
from tapas.protos import interaction_pb2
from tapas.utils import number_annotation_utils
from tapas.scripts import prediction_utils

# Load checkpoint for prediction

Here's the prediction code, which will create and `interaction_pb2.Interaction` protobuf object, which is the datastructure we use to store examples, and then call the prediction script.

In [7]:
os.makedirs('results/sqa/tf_examples', exist_ok=True)
os.makedirs('results/sqa/model', exist_ok=True)
with open('results/sqa/model/checkpoint', 'w') as f:
  f.write('model_checkpoint_path: "model.ckpt-0"')
for suffix in ['.data-00000-of-00001', '.index', '.meta']:
  shutil.copyfile(f'tapas_sqa_base/model.ckpt{suffix}', f'results/sqa/model/model.ckpt-0{suffix}')

In [8]:
max_seq_length = 512
vocab_file = "tapas_sqa_base/vocab.txt"
config = tf_example_utils.ClassifierConversionConfig(
    vocab_file=vocab_file,
    max_seq_length=max_seq_length,
    max_column_id=max_seq_length,
    max_row_id=max_seq_length,
    strip_column_names=False,
    add_aggregation_candidates=False,
)
converter = tf_example_utils.ToClassifierTensorflowExample(config)

def convert_interactions_to_examples(tables_and_queries):
  """Calls Tapas converter to convert interaction to example."""
  for idx, (table, queries) in enumerate(tables_and_queries):
    interaction = interaction_pb2.Interaction()
    for position, query in enumerate(queries):
      question = interaction.questions.add()
      question.original_text = query
      question.id = f"{idx}-0_{position}"
    for header in table[0]:
      interaction.table.columns.add().text = header
    for line in table[1:]:
      row = interaction.table.rows.add()
      for cell in line:
        row.cells.add().text = cell
    number_annotation_utils.add_numeric_values(interaction)
    for i in range(len(interaction.questions)):
      try:
        yield converter.convert(interaction, i)
      except ValueError as e:
        print(f"Can't convert interaction: {interaction.id} error: {e}")
        
def write_tf_example(filename, examples):
  with tf.io.TFRecordWriter(filename) as writer:
    for example in examples:
      writer.write(example.SerializeToString())

def predict(table_data, queries):
  table = [list(map(lambda s: s.strip(), row.split("|"))) 
           for row in table_data.split("\n") if row.strip()]
  examples = convert_interactions_to_examples([(table, queries)])
  write_tf_example("results/sqa/tf_examples/test.tfrecord", examples)
  write_tf_example("results/sqa/tf_examples/random-split-1-dev.tfrecord", [])
  
  ! python tapas/tapas/run_task_main.py \
    --task="SQA" \
    --output_dir="results" \
    --noloop_predict \
    --test_batch_size={len(queries)} \
    --tapas_verbosity="ERROR" \
    --compression_type= \
    --init_checkpoint="tapas_sqa_base/model.ckpt" \
    --bert_config_file="tapas_sqa_base/bert_config.json" \
    --mode="predict" 2> error


  results_path = "results/sqa/model/test_sequence.tsv"
  all_coordinates = []
  df = pd.DataFrame(table[1:], columns=table[0])
  display(IPython.display.HTML(df.to_html(index=False)))
  print()
  with open(results_path) as csvfile:
    reader = csv.DictReader(csvfile, delimiter='\t')
    for row in reader:
      coordinates = prediction_utils.parse_coordinates(row["answer_coordinates"])
      all_coordinates.append(coordinates)
      answers = ', '.join([table[row + 1][col] for row, col in coordinates])
      position = int(row['position'])
      print(">", queries[position])
      print(answers)
  return all_coordinates

# Predict

In [9]:
# Example nu-1000-0
result = predict("""
Pos | No | Driver               | Team                           | Laps | Time/Retired | Grid | Points
1   | 32 | Patrick Carpentier   | Team Player's                  | 87   | 1:48:11.023  | 1    | 22    
2   | 1  | Bruno Junqueira      | Newman/Haas Racing             | 87   | +0.8 secs    | 2    | 17    
3   | 3  | Paul Tracy           | Team Player's                  | 87   | +28.6 secs   | 3    | 14
4   | 9  | Michel Jourdain, Jr. | Team Rahal                     | 87   | +40.8 secs   | 13   | 12
5   | 34 | Mario Haberfeld      | Mi-Jack Conquest Racing        | 87   | +42.1 secs   | 6    | 10
6   | 20 | Oriol Servia         | Patrick Racing                 | 87   | +1:00.2      | 10   | 8 
7   | 51 | Adrian Fernandez     | Fernandez Racing               | 87   | +1:01.4      | 5    | 6
8   | 12 | Jimmy Vasser         | American Spirit Team Johansson | 87   | +1:01.8      | 8    | 5
9   | 7  | Tiago Monteiro       | Fittipaldi-Dingman Racing      | 86   | + 1 Lap      | 15   | 4
10  | 55 | Mario Dominguez      | Herdez Competition             | 86   | + 1 Lap      | 11   | 3
11  | 27 | Bryan Herta          | PK Racing                      | 86   | + 1 Lap      | 12   | 2
12  | 31 | Ryan Hunter-Reay     | American Spirit Team Johansson | 86   | + 1 Lap      | 17   | 1
13  | 19 | Joel Camathias       | Dale Coyne Racing              | 85   | + 2 Laps     | 18   | 0
14  | 33 | Alex Tagliani        | Rocketsports Racing            | 85   | + 2 Laps     | 14   | 0
15  | 4  | Roberto Moreno       | Herdez Competition             | 85   | + 2 Laps     | 9    | 0
16  | 11 | Geoff Boss           | Dale Coyne Racing              | 83   | Mechanical   | 19   | 0
17  | 2  | Sebastien Bourdais   | Newman/Haas Racing             | 77   | Mechanical   | 4    | 0
18  | 15 | Darren Manning       | Walker Racing                  | 12   | Mechanical   | 7    | 0
19  | 5  | Rodolfo Lavin        | Walker Racing                  | 10   | Mechanical   | 16   | 0
""", ["what were the drivers names?",
      "of these, which points did patrick carpentier and bruno junqueira score?",
      "who scored higher?",])

is_built_with_cuda: True
is_gpu_available: False
GPUs: []
Training or predicting ...
Evaluation finished after training step 0.


Pos,No,Driver,Team,Laps,Time/Retired,Grid,Points
1,32,Patrick Carpentier,Team Player's,87,1:48:11.023,1,22
2,1,Bruno Junqueira,Newman/Haas Racing,87,+0.8 secs,2,17
3,3,Paul Tracy,Team Player's,87,+28.6 secs,3,14
4,9,"Michel Jourdain, Jr.",Team Rahal,87,+40.8 secs,13,12
5,34,Mario Haberfeld,Mi-Jack Conquest Racing,87,+42.1 secs,6,10
6,20,Oriol Servia,Patrick Racing,87,+1:00.2,10,8
7,51,Adrian Fernandez,Fernandez Racing,87,+1:01.4,5,6
8,12,Jimmy Vasser,American Spirit Team Johansson,87,+1:01.8,8,5
9,7,Tiago Monteiro,Fittipaldi-Dingman Racing,86,+ 1 Lap,15,4
10,55,Mario Dominguez,Herdez Competition,86,+ 1 Lap,11,3



> what were the drivers names?
Bruno Junqueira, Joel Camathias, Michel Jourdain, Jr., Alex Tagliani, Tiago Monteiro, Geoff Boss, Rodolfo Lavin, Mario Dominguez, Ryan Hunter-Reay, Sebastien Bourdais, Darren Manning, Adrian Fernandez, Paul Tracy, Mario Haberfeld, Roberto Moreno, Oriol Servia, Patrick Carpentier, Jimmy Vasser, Bryan Herta
> of these, which points did patrick carpentier and bruno junqueira score?
22, 17
> who scored higher?
Patrick Carpentier


In [10]:
# Example nu-1000-0
result = predict("""
Pos | No | Driver               | Team                           | Laps | Time/Retired | Grid | Points
1   | 32 | Patrick Carpentier   | Team Player's                  | 87   | 1:48:11.023  | 1    | 22    
2   | 1  | Bruno Junqueira      | Newman/Haas Racing             | 87   | +0.8 secs    | 2    | 17    
3   | 3  | Paul Tracy           | Team Player's                  | 87   | +28.6 secs   | 3    | 14
4   | 9  | Michel Jourdain, Jr. | Team Rahal                     | 87   | +40.8 secs   | 13   | 12
5   | 34 | Mario Haberfeld      | Mi-Jack Conquest Racing        | 87   | +42.1 secs   | 6    | 10
6   | 20 | Oriol Servia         | Patrick Racing                 | 87   | +1:00.2      | 10   | 8 
7   | 51 | Adrian Fernandez     | Fernandez Racing               | 87   | +1:01.4      | 5    | 6
8   | 12 | Jimmy Vasser         | American Spirit Team Johansson | 87   | +1:01.8      | 8    | 5
9   | 7  | Tiago Monteiro       | Fittipaldi-Dingman Racing      | 86   | + 1 Lap      | 15   | 4
10  | 55 | Mario Dominguez      | Herdez Competition             | 86   | + 1 Lap      | 11   | 3
11  | 27 | Bryan Herta          | PK Racing                      | 86   | + 1 Lap      | 12   | 2
12  | 31 | Ryan Hunter-Reay     | American Spirit Team Johansson | 86   | + 1 Lap      | 17   | 1
13  | 19 | Joel Camathias       | Dale Coyne Racing              | 85   | + 2 Laps     | 18   | 0
14  | 33 | Alex Tagliani        | Rocketsports Racing            | 85   | + 2 Laps     | 14   | 0
15  | 4  | Roberto Moreno       | Herdez Competition             | 85   | + 2 Laps     | 9    | 0
16  | 11 | Geoff Boss           | Dale Coyne Racing              | 83   | Mechanical   | 19   | 0
17  | 2  | Sebastien Bourdais   | Newman/Haas Racing             | 77   | Mechanical   | 4    | 0
18  | 15 | Darren Manning       | Walker Racing                  | 12   | Mechanical   | 7    | 0
19  | 5  | Rodolfo Lavin        | Walker Racing                  | 10   | Mechanical   | 16   | 0
""", [ "what were the teams names?",
      "How many drivers have entered the race?",
      "Who is the winning driver?",
      "What is the winning car number?",
      "Who is the second player?",
      "Who is the 7th place driver?",
      "Who is the retired driver?",
      "What are the team drivers for Team Player's?",
      "What is the car number of the machine that was number one?",
      "Who is the winner"])

is_built_with_cuda: True
is_gpu_available: False
GPUs: []
Training or predicting ...
Evaluation finished after training step 0.


Pos,No,Driver,Team,Laps,Time/Retired,Grid,Points
1,32,Patrick Carpentier,Team Player's,87,1:48:11.023,1,22
2,1,Bruno Junqueira,Newman/Haas Racing,87,+0.8 secs,2,17
3,3,Paul Tracy,Team Player's,87,+28.6 secs,3,14
4,9,"Michel Jourdain, Jr.",Team Rahal,87,+40.8 secs,13,12
5,34,Mario Haberfeld,Mi-Jack Conquest Racing,87,+42.1 secs,6,10
6,20,Oriol Servia,Patrick Racing,87,+1:00.2,10,8
7,51,Adrian Fernandez,Fernandez Racing,87,+1:01.4,5,6
8,12,Jimmy Vasser,American Spirit Team Johansson,87,+1:01.8,8,5
9,7,Tiago Monteiro,Fittipaldi-Dingman Racing,86,+ 1 Lap,15,4
10,55,Mario Dominguez,Herdez Competition,86,+ 1 Lap,11,3



> what were the teams names?
American Spirit Team Johansson, Fittipaldi-Dingman Racing, Newman/Haas Racing, Walker Racing, Team Rahal, Rocketsports Racing, Dale Coyne Racing, Newman/Haas Racing, Herdez Competition, American Spirit Team Johansson, Fernandez Racing, Walker Racing, Team Player's, Mi-Jack Conquest Racing, Herdez Competition, Team Player's, PK Racing, Dale Coyne Racing, Patrick Racing
> How many drivers have entered the race?
Bruno Junqueira, Joel Camathias, Michel Jourdain, Jr., Alex Tagliani, Tiago Monteiro, Geoff Boss, Rodolfo Lavin, Mario Dominguez, Ryan Hunter-Reay, Sebastien Bourdais, Darren Manning, Adrian Fernandez, Paul Tracy, Mario Haberfeld, Roberto Moreno, Oriol Servia, Patrick Carpentier, Jimmy Vasser, Bryan Herta
> Who is the winning driver?
Mario Dominguez
> What is the winning car number?
3
> Who is the second player?
Bruno Junqueira
> Who is the 7th place driver?
Adrian Fernandez
> Who is the retired driver?
Darren Manning
> What are the team drivers for T

In [11]:
result = predict("""
Pos | Student               | Japanese | mathematics | science | social studies | art  | music | physical education | homemaking | English
1   | Patrick Carpentier     | 100      | 100          | 　98    | 100          | 100 | 100  | 100               | 100         | 100 
2   | Bruno Junqueira       | 100      | 100          | 100    | 100          | 100 | 100  | 100               | 100         | 100 
3   | Paul Tracy            | 100      | 100          | 100    | 100          |   82 |  92  | 100               | 100         | 100 
4   | Michel Jourdain, Jr.    | 100      | 100          | 100    | 100          | 100 | 100  | 100               | 100         | 100 
5   | Mario Haberfeld       | 100      | 100          | 100    | 100          | 100 | 100  | 100               | 100         |   23 
6   | Oriol Servia           | 100      | 100          | 100    | 100          | 100 | 100  | 100               | 100         | 100 
7   | Adrian Fernandez     | 100      | 100          | 100    | 100          | 100 | 100  | 100               | 100         | 100 
8   | Jimmy Vasser         | 100      | 100          | 100    | 100          | 100 | 100  | 100               | 100         |  23 
9   | Tiago Monteiro        | 100      | 100          | 100    | 100          | 100 | 100  | 100               | 100         | 100 
10  | Mario Dominguez     | 　　32      | 100          | 100    | 100          | 100 | 100  | 100               | 100         | 100 
""", [ "What is your score in Japanese, Patrick Carpentier?",
      "What is your score in science, Patrick Carpentier?",
      "What is your score in Japanese, Mario Dominguez?",
      "What is your score in art and music, Paul Tracy?",
      "Who is a student with a score of 23 in English?"])

is_built_with_cuda: True
is_gpu_available: False
GPUs: []
Training or predicting ...
Evaluation finished after training step 0.


Pos,Student,Japanese,mathematics,science,social studies,art,music,physical education,homemaking,English
1,Patrick Carpentier,100,100,98,100,100,100,100,100,100
2,Bruno Junqueira,100,100,100,100,100,100,100,100,100
3,Paul Tracy,100,100,100,100,82,92,100,100,100
4,"Michel Jourdain, Jr.",100,100,100,100,100,100,100,100,100
5,Mario Haberfeld,100,100,100,100,100,100,100,100,23
6,Oriol Servia,100,100,100,100,100,100,100,100,100
7,Adrian Fernandez,100,100,100,100,100,100,100,100,100
8,Jimmy Vasser,100,100,100,100,100,100,100,100,23
9,Tiago Monteiro,100,100,100,100,100,100,100,100,100
10,Mario Dominguez,32,100,100,100,100,100,100,100,100



> What is your score in Japanese, Patrick Carpentier?
100
> What is your score in science, Patrick Carpentier?
98
> What is your score in Japanese, Mario Dominguez?
32
> What is your score in art and music, Paul Tracy?
82
> Who is a student with a score of 23 in English?
Mario Haberfeld, Jimmy Vasser


In [12]:
result = predict("""
Pos | Student           | Japanese | mathematics | science | social studies | art  | music | physical education | homemaking | English
1   | 瀧澤 伸            | 100      | 100          | 　98    | 100          | 100 | 100  | 100               | 100         | 100 
2   | たきざわ しん       | 100      | 100          | 100    | 100          | 100 | 100  | 100               | 100         | 100 
3   | 瀧澤 ひな          | 100      | 100          | 100    | 100          |   82 |  92  | 100               | 100         | 100 
4   | フグ田 サザエ       | 100      | 100          | 100    | 100          | 100 | 100  | 100               | 100         | 100 
5   | 磯野 カツオ         | 100      | 100          | 100    | 100          | 100 | 100  | 100               | 100         |   23 
6   | 磯野 ワカメ         | 100      | 100          | 100    | 100          | 100 | 100  | 100               | 100         | 100 
7   | フグ田 マスオ      | 100      | 100          | 100    | 100          | 100 | 100  | 100               | 100         | 100 
8   | 磯野 波平         | 100      | 100          | 100    | 100          | 100 | 100  | 100               | 100         |  23 
9   | 磯野 フネ         | 100      | 100          | 100    | 100          | 100 | 100  | 100               | 100         | 100 
10  | フグ田 タラオ     | 　　32      | 100          | 100    | 100          | 100 | 100  | 100               | 100         | 100 
""", [ "What is your score in Japanese, 瀧澤 伸?",
      "What is your score in science, 瀧澤 伸?",
      "What is your score in Japanese, フグ田 タラオ?",
      "What is your score in art and music, 瀧澤 ひな?",
      "Who is a student with a score of 23 in English?"])

is_built_with_cuda: True
is_gpu_available: False
GPUs: []
Training or predicting ...
Evaluation finished after training step 0.


Pos,Student,Japanese,mathematics,science,social studies,art,music,physical education,homemaking,English
1,瀧澤 伸,100,100,98,100,100,100,100,100,100
2,たきざわ しん,100,100,100,100,100,100,100,100,100
3,瀧澤 ひな,100,100,100,100,82,92,100,100,100
4,フグ田 サザエ,100,100,100,100,100,100,100,100,100
5,磯野 カツオ,100,100,100,100,100,100,100,100,23
6,磯野 ワカメ,100,100,100,100,100,100,100,100,100
7,フグ田 マスオ,100,100,100,100,100,100,100,100,100
8,磯野 波平,100,100,100,100,100,100,100,100,23
9,磯野 フネ,100,100,100,100,100,100,100,100,100
10,フグ田 タラオ,32,100,100,100,100,100,100,100,100



> What is your score in Japanese, 瀧澤 伸?
100
> What is your score in science, 瀧澤 伸?
98
> What is your score in Japanese, フグ田 タラオ?
32
> What is your score in art and music, 瀧澤 ひな?
82
> Who is a student with a score of 23 in English?
磯野 カツオ


In [13]:
result = predict("""
Pos | 学生              | 国語       | 算数          | 理科    | 社会         | 美術 | 音楽   | 体育              | 家庭科       | 英語
1   | 瀧澤 伸            | 100      | 100          | 　98    | 100          | 100 | 100  | 100               | 100         | 100 
2   | たきざわ しん       | 100      | 100          | 100    | 100          | 100 | 100  | 100               | 100         | 100 
3   | 瀧澤 ひな          | 100      | 100          | 100    | 100          |   82 |  92  | 100               | 100         | 100 
4   | フグ田 サザエ       | 100      | 100          | 100    | 100          | 100 | 100  | 100               | 100         | 100 
5   | 磯野 カツオ         | 100      | 100          | 100    | 100          | 100 | 100  | 100               | 100         |   23 
6   | 磯野 ワカメ         | 100      | 100          | 100    | 100          | 100 | 100  | 100               | 100         | 100 
7   | フグ田 マスオ      | 100      | 100          | 100    | 100          | 100 | 100  | 100               | 100         | 100 
8   | 磯野 波平         | 100      | 100          | 100    | 100          | 100 | 100  | 100               | 100         |  23 
9   | 磯野 フネ         | 100      | 100          | 100    | 100          | 100 | 100  | 100               | 100         | 100 
10  | フグ田 タラオ     | 　　32      | 100          | 100    | 100          | 100 | 100  | 100               | 100         | 100 
""", [ "What is your score in 国語, 瀧澤 伸?",
      "What is your score in 理科, 瀧澤 伸?",
      "What is your score in 国語, フグ田 タラオ?",
      "What is your score in 美術 and 音楽, 瀧澤 ひな?",
      "Who is a 学生 with a score of 23 in English?"])

is_built_with_cuda: True
is_gpu_available: False
GPUs: []
Training or predicting ...
Evaluation finished after training step 0.


Pos,学生,国語,算数,理科,社会,美術,音楽,体育,家庭科,英語
1,瀧澤 伸,100,100,98,100,100,100,100,100,100
2,たきざわ しん,100,100,100,100,100,100,100,100,100
3,瀧澤 ひな,100,100,100,100,82,92,100,100,100
4,フグ田 サザエ,100,100,100,100,100,100,100,100,100
5,磯野 カツオ,100,100,100,100,100,100,100,100,23
6,磯野 ワカメ,100,100,100,100,100,100,100,100,100
7,フグ田 マスオ,100,100,100,100,100,100,100,100,100
8,磯野 波平,100,100,100,100,100,100,100,100,23
9,磯野 フネ,100,100,100,100,100,100,100,100,100
10,フグ田 タラオ,32,100,100,100,100,100,100,100,100



> What is your score in 国語, 瀧澤 伸?
100
> What is your score in 理科, 瀧澤 伸?
98
> What is your score in 国語, フグ田 タラオ?
32, 100
> What is your score in 美術 and 音楽, 瀧澤 ひな?
82, 100
> Who is a 学生 with a score of 23 in English?
磯野 カツオ


In [14]:
result = predict("""
Pos | 学生              | 国語       | 算数          | 理科    | 社会         | 美術 | 音楽   | 体育              | 家庭科       | 英語
1   | 瀧澤 伸            | 100      | 100          | 　98    | 100          | 100 | 100  | 100               | 100         | 100 
2   | たきざわ しん       | 100      | 100          | 100    | 100          | 100 | 100  | 100               | 100         | 100 
3   | 瀧澤 ひな          | 100      | 100          | 100    | 100          |   82 |  92  | 100               | 100         | 100 
4   | フグ田 サザエ       | 100      | 100          | 100    | 100          | 100 | 100  | 100               | 100         | 100 
5   | 磯野 カツオ         | 100      | 100          | 100    | 100          | 100 | 100  | 100               | 100         |   23 
6   | 磯野 ワカメ         | 100      | 100          | 100    | 100          | 100 | 100  | 100               | 100         | 100 
7   | フグ田 マスオ      | 100      | 100          | 100    | 100          | 100 | 100  | 100               | 100         | 100 
8   | 磯野 波平         | 100      | 100          | 100    | 100          | 100 | 100  | 100               | 100         |  23 
9   | 磯野 フネ         | 100      | 100          | 100    | 100          | 100 | 100  | 100               | 100         | 100 
10  | フグ田 タラオ     | 　　32      | 100          | 100    | 100          | 100 | 100  | 100               | 100         | 100 
""", [ "What is your score in 国語, 瀧澤 伸?",
      "What is your score in 理科, 瀧澤 伸?",
      "What is your score in 国語, フグ田 タラオ?",
      "What is your score in 美術 and 音楽, 瀧澤 ひな?",
      "Who is a 学生 with a score of 23 in English?"])

is_built_with_cuda: True
is_gpu_available: False
GPUs: []
Training or predicting ...
Evaluation finished after training step 0.


Pos,学生,国語,算数,理科,社会,美術,音楽,体育,家庭科,英語
1,瀧澤 伸,100,100,98,100,100,100,100,100,100
2,たきざわ しん,100,100,100,100,100,100,100,100,100
3,瀧澤 ひな,100,100,100,100,82,92,100,100,100
4,フグ田 サザエ,100,100,100,100,100,100,100,100,100
5,磯野 カツオ,100,100,100,100,100,100,100,100,23
6,磯野 ワカメ,100,100,100,100,100,100,100,100,100
7,フグ田 マスオ,100,100,100,100,100,100,100,100,100
8,磯野 波平,100,100,100,100,100,100,100,100,23
9,磯野 フネ,100,100,100,100,100,100,100,100,100
10,フグ田 タラオ,32,100,100,100,100,100,100,100,100



> How does 瀧澤 伸 score in 国語?
100
> What is your score in 理科, 瀧澤 伸?
98
> What is your score in 国語, フグ田 タラオ?
32, 100
> What is your score in 美術 and 音楽, 瀧澤 ひな?
82, 100
> Who is a 学生 with a score of 23 in English?
磯野 カツオ


In [38]:
result = predict("""
科目 | 学生              | 国語       | 算数          | 理科    | 社会         | 美術 | 音楽   | 体育              | 家庭科       | 英語
1   | 伸            | 100      | 100          | 　98    | 100          | 100 | 100  | 100               | 100         | 100 
2   | しん       | 100      | 100          | 100    | 100          | 100 | 100  | 100               | 100         | 100 
3   | ひな          | 100      | 100          | 100    | 100          |   82 |  92  | 100               | 100         | 100 
4   | サザエ       | 100      | 100          | 100    | 100          | 100 | 100  | 100               | 100         | 100 
5   | カツオ         | 100      | 100          | 100    | 100          | 100 | 100  | 100               | 100         |   23 
6   | ワカメ         | 100      | 100          | 100    | 100          | 100 | 100  | 100               | 100         | 100 
7   | マスオ      | 100      | 100          | 100    | 100          | 100 | 100  | 100               | 100         | 100 
8   | 波平         | 100      | 100          | 100    | 100          | 100 | 100  | 100               | 100         |  23 
9   | フネ         | 100      | 100          | 100    | 100          | 100 | 100  | 100               | 100         | 100 
10  | タラオ     | 　　32      | 100          | 100    | 100          | 100 | 100  | 100               | 100         | 100 
""", [ "How does 伸 score in 国語?",
      "How does 伸 score in 理科?",
      "Who has the lowest score in 美術?",
      "Who has the lowest score in 英語?",
      "What is ひな's highest score?",
      "What is ひな's lowest score?",
      "How does 波平's score in 英語?",
      "How does タラオ score in 国語?",
      "Who are the 学生?",
      "What 学生 doesn't score 100 in 理科?",
      "What about 学生 who scored a 100 in 国語?",
      "What's 波平's score??",
      "Who are the 学生 who scored 50 or more on the 国語?",
      ])

is_built_with_cuda: True
is_gpu_available: False
GPUs: []
Training or predicting ...
Evaluation finished after training step 0.


科目,学生,国語,算数,理科,社会,美術,音楽,体育,家庭科,英語
1,伸,100,100,98,100,100,100,100,100,100
2,しん,100,100,100,100,100,100,100,100,100
3,ひな,100,100,100,100,82,92,100,100,100
4,サザエ,100,100,100,100,100,100,100,100,100
5,カツオ,100,100,100,100,100,100,100,100,23
6,ワカメ,100,100,100,100,100,100,100,100,100
7,マスオ,100,100,100,100,100,100,100,100,100
8,波平,100,100,100,100,100,100,100,100,23
9,フネ,100,100,100,100,100,100,100,100,100
10,タラオ,32,100,100,100,100,100,100,100,100



> How does 伸 score in 国語?
100
> How does 伸 score in 理科?
98
> Who has the lowest score in 美術?

> Who has the lowest score in 英語?
波平
> What is ひな's highest score?

> What is ひな's lowest score?
3
> How does 波平's score in 英語?
23
> How does タラオ score in 国語?
32
> Who are the 学生?
伸, タラオ, 波平, フネ, マスオ, サザエ, ひな, ワカメ, カツオ, しん
> What 学生 doesn't score 100 in 理科?
波平
> What about 学生 who scored a 100 in 国語?

> What's 波平's score??
8
> Who are the 学生 who scored 50 or more on the 国語?
マスオ, しん, 波平
